<a href="https://colab.research.google.com/github/myazann/Lung_Cancer/blob/main/Data_Preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git init
!git pull https://github.com/myazann/TCIA-API-SDK.git

!cp -r /content/tcia-rest-client-python/src/* /content

!rm -rf tcia-rest-client-java/
!rm -rf tcia-rest-client-python/

!cp /content/drive/MyDrive/Annotation.zip /content
!cp /content/drive/MyDrive/images.zip /content
!unzip Annotation.zip > /dev/null
!rm Annotation.zip

!pip install xmltodict
!rm -rf sample_data
!rm sample.py
!rm README.md

!pip install pip install pandasql
from tciaclient import TCIAClient
import json
import os
import numpy as np
from xml.etree import cElementTree as ElementTree
import xmltodict
import urllib.request, urllib.error, urllib.parse
import subprocess
import pandas as pd
import pandasql as ps

tc = TCIAClient(baseUrl="https://services.cancerimagingarchive.net/services/v4", resource = "TCIA")

##Get Annotations, Patients, Series ID's and SOP ID's

In [59]:
## Get bounding boxs associated with the images
folders = np.array(os.listdir("Annotation"))

bboxs = {}

for folder in folders:
  path = "Annotation/" + folder
  for img in os.listdir(path):
    file_path = path + "/" + img
    try:
      dt = xmltodict.parse(ElementTree.tostring(ElementTree.parse(file_path).getroot()))["annotation"]["object"]
      if isinstance(dt, list):
        box_list = []
        for box in dt:
          box_list.append(box)
      else:
        dt = dt["bndbox"]
      img_name = folder + "-" + img.split(".x")[0]
      bboxs[img_name] = dt
    except Exception as e:
      print(e)

with open('bboxs.json', "w") as json_file:
  json.dump(bboxs, json_file)


## Get the series id of each patient with the most images
clctn = "Lung-PET-CT-Dx"
series = json.loads(tc.get_series(collection = clctn, modality = "CT").read())
ptnt_to_series_ids = pd.DataFrame()

pt_id = []
series_id = []
img_count = []

for elem in series:
  try:
    if elem["BodyPartExamined"] == "CHEST":
      pt_id.append(elem["PatientID"].split("-")[1])
      series_id.append(elem["SeriesInstanceUID"])
      img_count.append(elem["ImageCount"])
  except Exception as e:
    print(e)

ptnt_to_series_ids["Patient_ID"] = pt_id
ptnt_to_series_ids["Series_ID"] = series_id
ptnt_to_series_ids["Image_Count"] = img_count

## Pandas groupby don't work this way so I used an SQL statement
ptnt_to_series_ids = ps.sqldf("""SELECT Patient_ID,MAX(Image_Count) Image_Count ,
MAX(Series_ID) Series_ID FROM ptnt_to_series_ids
GROUP BY Patient_ID""")

ptnt_to_series_ids = dict((p,s) for p,s in ptnt_to_series_ids[["Patient_ID","Series_ID"]].values)

## Get the image id's (SOP) in each series

for ptnt, series in ptnt_to_series_ids.items():
  sop_to_series_ids = {}
  sop = json.loads(tc.get_SOP_instance(series).read())

  sop_to_series_ids[series] = [s["SOPInstanceUID"] for s in sop]
  ptnt_to_series_ids[ptnt] = sop_to_series_ids

In [70]:
np.sum(img_count)

104498

In [63]:
ptnts_with_bboxs = [box.split("-")[0] for box in bboxs.keys()]
imgs_with_bboxs = [box.split("-")[1] for box in bboxs.keys()]

ptnt_to_series_ids = dict((ptnt, ptnt_to_series_ids[ptnt]) for ptnt in ptnt_to_series_ids.keys() if ptnt in ptnts_with_bboxs)

for ptnt in ptnt_to_series_ids:
  series_id = list(ptnt_to_series_ids[ptnt].keys())[0]
  for img in ptnt_to_series_ids[ptnt][series_id]:
    if img not in imgs_with_bboxs:
      ptnt_to_series_ids[ptnt][series_id].remove(img)

with open('ptnt_to_series_ids.json', 'w') as json_file:
  json.dump(ptnt_to_series_ids, json_file)

In [64]:
i = 0
for ptnt in ptnt_to_series_ids:
  series_id = list(ptnt_to_series_ids[ptnt].keys())[0]
  for img in ptnt_to_series_ids[ptnt][series_id]:
    i += 1

In [ ]:
bboxs

## Download images

In [37]:
counter = 0

if not os.path.exists("images"):
  os.mkdir("images")
else:
  downloaded_images = []
  for ptnt in os.listdir("images"):
    for img in os.listdir("images/" + ptnt):
      downloaded_images.append(img)
  counter = len(downloaded_images)
  downloaded_images = [img.split(".d")[0] for img in downloaded_images]


for ptnt in ptnt_to_series_ids:

  series_id = list(ptnt_to_series_ids[ptnt].keys())[0]
  print(ptnt)

  if not os.path.exists("./images/" + ptnt):
    os.mkdir("./images/" + ptnt)

    for img in ptnt_to_series_ids[ptnt][series_id]:
      download_path = "./images/" + ptnt + "/" + img + ".dcm"
      tc.get_single_image(series_id, img, downloadPath = download_path)
      counter += 1
      print(counter)

      if counter % 1000 == 0:
        subprocess.call(["zip","-r","images.zip","images",">", "/dev/null"])
        subprocess.call(["mv","images.zip","./drive/MyDrive/"])   

  else:

    for img in ptnt_to_series_ids[ptnt][series_id]:

      if img not in downloaded_images:
        download_path = "./images/" + ptnt + "/" + img + ".dcm"
        tc.get_single_image(series_id, img, downloadPath = download_path)
        counter += 1
        print(counter)

      if counter % 1000 == 0:
        subprocess.call(["zip","-r","images.zip","images",">", "/dev/null"])
        subprocess.call(["mv","images.zip","./drive/MyDrive/"])      

  print(counter)

subprocess.call(["zip","-r","images.zip","images",">", "/dev/null"])
subprocess.call(["mv","images.zip","./drive/MyDrive/"])

3374
3375


KeyboardInterrupt: ignored

## Split files into train and test (%80-%20). A patient who is in train cannot be in test and classes should be distributed equally in both sets

In [ ]:
ptnt_classes = {}

ptnt_classes["A"] = [k for k in ptnt_to_series_ids.keys() if "A" in k]
ptnt_classes["B"] = [k for k in ptnt_to_series_ids.keys() if "B" in k]
ptnt_classes["E"] = [k for k in ptnt_to_series_ids.keys() if "E" in k]
ptnt_classes["G"] = [k for k in ptnt_to_series_ids.keys() if "G" in k]

train_ptnts = []
val_ptnts = []

for cls in ptnt_classes.keys():
  permutated_list = list(np.random.permutation(np.array(ptnt_classes[cls])))
  val_len = int(len(permutated_list)*0.2)

  val_ptnts.extend(permutated_list[0:val_len])
  train_ptnts.extend(permutated_list[val_len:])
  

!mkdir /content/lung_ct_train
!mkdir /content/lung_ct_val


for ptnt in os.listdir("images"):
  cur_path = "./images/" + ptnt
  sop_name = img.split(".d")[0]
  
  if ptnt in train_ptnts:
    os.mkdir("/content/lung_ct_train/" + ptnt)
    new_path = "lung_ct_train/" + ptnt
  else:
    os.mkdir("/content/lung_ct_val/" + ptnt)
    new_path = "lung_ct_val/" + ptnt
  os.rename(cur_path, new_path)


subprocess.call(["zip","-r","lung_ct_train.zip","lung_ct_train",">", "/dev/null"])
subprocess.call(["mv","lung_ct_train.zip","./drive/MyDrive/"])

subprocess.call(["zip","-r","lung_ct_val.zip","lung_ct_val",">", "/dev/null"])
subprocess.call(["mv","lung_ct_val.zip","./drive/MyDrive/"])